In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding

In [2]:
import numpy as np

In [3]:
# define data
train_data = ['very well done',
        'good work done by the team',
        'you took great effort',
        'nice work done by the members',
        'excellent efforts on part of everyone',
        'poor effort',
        'not good',
        'poor work',
        'could have done better']
# define sentiment labels
labels = np.array([1,1,1,1,1,0,0,0,0])

In [4]:
# lowercase all words and create a vocabulary
all_eng_words=set()
for eng in train_data:
    for word in eng.split():
            all_eng_words.add(word.lower())
all_eng_words

{'better',
 'by',
 'could',
 'done',
 'effort',
 'efforts',
 'everyone',
 'excellent',
 'good',
 'great',
 'have',
 'members',
 'nice',
 'not',
 'of',
 'on',
 'part',
 'poor',
 'team',
 'the',
 'took',
 'very',
 'well',
 'work',
 'you'}

In [5]:
num_encoder_tokens = len(all_eng_words)
num_encoder_tokens

25

In [6]:
num_encoder_tokens += 1 #for zero padding
num_encoder_tokens

26

In [7]:
input_words = sorted(list(all_eng_words))
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
reverse_input_index = dict((i, word) for word, i in input_token_index.items())

#### The embedding layer takes at least 2 arguments
#### no of words in vocab + 1 (for padding)
#### and the dimensionality of the embedding
#### This layer can only be used as the first layer in a model.

In [8]:
max_length_src=max(train_data, key=len)
max_length_src

'excellent efforts on part of everyone'

In [9]:
max_length_src=len(max_length_src.split())
max_length_src

6

In [10]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Flatten
from tensorflow.keras.models import Sequential

In [11]:
#emb_dim = 5
# define the model
model = Sequential()
model.add(Embedding(num_encoder_tokens, 
                    5, 
                    input_length=max_length_src))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 6, 5)              130       
_________________________________________________________________
flatten (Flatten)            (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 31        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
#parameters of embedding layer = token_size*no of word embeddiing dimensions

In [13]:
encoder_input_data = np.zeros((9, max_length_src),dtype='float32')
for i, input_text in enumerate(train_data):
         for t, word in enumerate(input_text.split()):
                # encoder input seq
                encoder_input_data[i, t] = input_token_index[word.lower()] 

In [14]:
encoder_input_data

array([[22., 23.,  4.,  0.,  0.,  0.],
       [ 9., 24.,  4.,  2., 20., 19.],
       [25., 21., 10.,  5.,  0.,  0.],
       [13., 24.,  4.,  2., 20., 12.],
       [ 8.,  6., 16., 17., 15.,  7.],
       [18.,  5.,  0.,  0.,  0.,  0.],
       [14.,  9.,  0.,  0.,  0.,  0.],
       [18., 24.,  0.,  0.,  0.,  0.],
       [ 3., 11.,  4.,  1.,  0.,  0.]], dtype=float32)

In [15]:
model.fit(encoder_input_data, labels, epochs=50)

Epoch 1/50
1/1 [==============================] - 0s 0s/step - loss: 0.6957 - accuracy: 0.4444
Epoch 2/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6934 - accuracy: 0.5556
Epoch 3/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6911 - accuracy: 0.5556
Epoch 4/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6888 - accuracy: 0.5556
Epoch 5/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6865 - accuracy: 0.6667
Epoch 6/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6842 - accuracy: 0.7778
Epoch 7/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6819 - accuracy: 0.7778
Epoch 8/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6796 - accuracy: 0.7778
Epoch 9/50
1/1 [==============================] - 0s 0s/step - loss: 0.6773 - accuracy: 0.7778
Epoch 10/50
1/1 [==============================] - 0s 15ms/step - loss: 0.6750 - accuracy: 0.7778
Epoch 11/50
1/1 [=======================

In [16]:
encoder_input_data.shape

(9, 6)

In [17]:
test_data ='not done well'
test_data = np.array([[14., 4.,23., 0. ,0. ,0.]]).reshape(1,6)
model.predict(test_data)

array([[0.47042567]], dtype=float32)

In [18]:
model.layers[0].get_weights()[0].shape

(26, 5)

In [19]:
model.layers[0].get_weights()[0]

array([[ 0.01833678, -0.04171934,  0.09368385,  0.03802868,  0.00325983],
       [ 0.02218266, -0.0977444 ,  0.0927205 , -0.0005423 ,  0.09035153],
       [-0.0228668 ,  0.00767461, -0.07337783, -0.02228965, -0.06538216],
       [ 0.00376928, -0.00826061,  0.06869004, -0.07292867, -0.0898812 ],
       [-0.04430699, -0.0801262 , -0.02910785, -0.071634  ,  0.06116445],
       [-0.08186093,  0.03130139, -0.08237974, -0.01509692,  0.08851887],
       [ 0.03175548, -0.06967256,  0.02768652, -0.0198629 , -0.00728823],
       [ 0.08193626,  0.0951513 , -0.05666602, -0.06466027,  0.06475899],
       [ 0.01560468,  0.04749871, -0.01863059,  0.01792593,  0.00734033],
       [-0.0885355 ,  0.07014767, -0.06155946,  0.06812058,  0.01431042],
       [-0.0586049 , -0.00215694, -0.05603223, -0.06199695,  0.046665  ],
       [-0.02168677,  0.04412618, -0.04789738,  0.05795659,  0.07937761],
       [ 0.05145985,  0.06986363, -0.08783805, -0.03640277,  0.03809496],
       [-0.01188653,  0.08303266, -0.0

In [20]:
from sklearn.metrics.pairwise import pairwise_distances